In [40]:
import pandas as pd
import numpy as np

# restaurants_limited_features
df_res = pd.read_csv("/Users/dima/code/Dimasaur/scorecast/data/restaurants_limited_features.csv")

# restaurant ids dataset
res_ids = pd.read_csv("/Users/dima/code/Dimasaur/scorecast/data/restaurants.csv")

# yelp general business dataset
business_json_path = "/Users/dima/code/Dimasaur/scorecast/data/yelp_academic_dataset_business.json"

#reading the json file
df_business_raw = pd.read_json(business_json_path, lines=True)

# creating a destination path
csv_file_path_review = "yelp_business_raw.csv"

#converting the output into a csv file
df_business_raw.to_csv(csv_file_path_review, index=False)



df_res['food_type_string'] = df_res['food_type'].apply(lambda x: str(x).strip('[]'))
df_res['food_type_string'] = df_res['food_type_string'].apply(lambda x: str(x).replace("'",""))

# replace the values in the food_types with the top - 20 cuisines

replace_dictionary = {
    "Pizza" : "Italian",
    "Burgers, Fast Food" : "Burgers",
    "Italian, Pizza" : "Italian",
    "American (New)" : "American",
    "Fast Food, Sandwiches" : "Sandwiches",
    "American (Traditional)" : "American",
    "Fast Food, Mexican" : "Mexican",
    "Coffee & Tea, Food" : "Cafes",
    "Japanese, Sushi Bars" : "Japanese",
    "American (Traditional), Diners" : "Diners",
    "Bakeries, Food" : "Bakery",
    "Delis, Sandwiches" : "Delis",
    "Mexican, Tex-Mex" : "Mexican",
    "American (Traditional), Burgers" : "American",
    "Sushi Bars" : "Japanese",
    "Chinese, Fast Food" : "Chinese",
    "American (Traditional), Cafes" : "American",
    "Mexican, Tacos" : "Mexican",
    "Pizza, Sandwiches" : "Sandwiches",
    "Food, Pizza" : "Italian",
    "American (Traditional), Seafood" : "American",
    "Greek, Mediterranean" : "Greek",
    "American (New), American (Traditional)" : "American",
    "Cajun/Creole, Seafood" : "Cajun/Creole"

}

df_res.food_type_string = df_res.food_type_string.replace(replace_dictionary)

# generate the clea array of the top 20 categories
df_food_t20 = list(df_res.food_type_string.value_counts(normalize=True).head(25).index)
df_food_t20.remove('O')
df_food_t20.remove('')
df_food_t20 = df_food_t20[:20]

# top 20 food types
np_food_20 = np.array([df_food_t20])


# filtering the the list of business to get the restaurants only
df_restaurants = df_business_raw[df_business_raw.business_id.isin(res_ids.business_id)].reset_index(drop=True)


df_restaurants.categories_clean = df_restaurants.categories.str.replace(", ",",")

df_restaurants['food_type_one'] = None

# function that extracts a category from a column in a dataframe if that category is present in a given list


def extract_category(categories, food_types):

    # Combine the lists after splitting by space and by comma
    possible_foods = categories.split(' ') + categories.split(',')
    # Remove any empty strings that might be created by split
    possible_foods = [food.strip() for food in possible_foods if food.strip()]

    for food in possible_foods:
        if food in food_types:
            return food
    return None

df_restaurants['food_type_one'] = df_restaurants['categories'].apply(lambda x: extract_category(x, df_food_t20))

states_to_exclude = ['XMS','CO','HI','MT','NC']

df_restaurants = df_restaurants[~df_restaurants['state'].isin(states_to_exclude)]


/var/folders/fd/585jf60x39vflmpy4gfz7yzh0000gn/T/ipykernel_79611/1900787865.py:5: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_res = pd.read_csv("/Users/dima/code/Dimasaur/scorecast/data/restaurants_limited_features.csv")
/var/folders/fd/585jf60x39vflmpy4gfz7yzh0000gn/T/ipykernel_79611/1900787865.py:73: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_restaurants.categories_clean = df_restaurants.categories.str.replace(", ",",")


In [41]:
df_restaurants.groupby(['state','food_type_one']).stars.mean().sort_values(ascending=False).tail(50)

state  food_type_one
AZ     Sandwiches       3.365613
AB     Mexican          3.361702
LA     Sandwiches       3.349359
IN     Sandwiches       3.342752
PA     Diners           3.340541
MO     Sandwiches       3.328502
DE     American         3.321429
FL     Chinese          3.320175
ID     Mexican          3.316000
NJ     Diners           3.297297
IL     Mexican          3.291209
AZ     Chinese          3.280576
TN     Italian          3.275000
MO     Mexican          3.258278
AB     Steakhouses      3.244186
IL     Italian          3.241667
AB     American         3.226316
TN     Chinese          3.223333
CA     Chicken Wings    3.187500
IL     Greek            3.166667
NV     Chinese          3.155844
DE     Chicken Wings    3.152174
       Chinese          3.149123
NV     Chicken Wings    3.144737
ID     Chinese          3.125000
AZ     Cajun/Creole     3.090909
IL     Sandwiches       3.086420
LA     Chicken Wings    3.042683
NJ     Chicken Wings    3.033333
AB     Burgers        

In [42]:
states_mean_review_scores = pd.DataFrame(df_restaurants.groupby(['state']).stars.mean().sort_values(ascending=False))
states_mean_review_scores.to_csv('/Users/dima/code/Dimasaur/scorecast/eda/states_mean_review_scores.csv')

In [43]:
unique_states = pd.DataFrame(df_restaurants['state'].unique(), columns=['state'])
unique_dtates = unique_states.state.str.upper()
unique_states = pd.DataFrame(unique_states.state.sort_values(ascending=True).reset_index(drop=True))

unique_cities = pd.DataFrame(df_restaurants['city'].unique(), columns=['city'])
unique_cities = pd.DataFrame(unique_cities.city.sort_values(ascending=True).reset_index(drop=True))


In [44]:
unique_cities.to_csv('cities.csv')
unique_states.to_csv('states.csv')

In [45]:
df_restaurants.city = df_restaurants.city.astype(str)
df_restaurants.state = df_restaurants.state.astype(str)

df_restaurants.city = df_restaurants.city.str.upper()
df_restaurants.state = df_restaurants.state.str.upper()

In [46]:
df_restaurants.state.value_counts()

state
PA    12641
FL     8731
TN     4352
MO     4247
IN     4150
LA     3640
NJ     3341
AZ     2671
AB     2410
NV     1673
ID     1302
CA     1161
IL      983
DE      961
Name: count, dtype: int64

In [47]:
state_city_dict = df_restaurants.groupby('state')['city'].apply(set).to_dict()
state_city_dict = {state: list(cities) for state, cities in state_city_dict.items()}
state_city_dict.keys()

dict_keys(['AB', 'AZ', 'CA', 'DE', 'FL', 'ID', 'IL', 'IN', 'LA', 'MO', 'NJ', 'NV', 'PA', 'TN'])

In [9]:
import json

with open('state_city_dict.json', 'w') as json_file:
    json.dump(state_city_dict, json_file, indent=4)


In [53]:
state_names = {
    'AB': 'ALBERTA',
    'AZ': 'ARIZONA',
    'CA': 'CALIFORNIA',
    'DE': 'DELAWARE',
    'FL': 'FLORIDA',
    'ID': 'IDAHO',
    'IL': 'ILLINOIS',
    'IN': 'INDIANA',
    'LA': 'LOUISIANA',
    'MO': 'MISSOURI',
    'NJ': 'NEW JERSEY',
    'NV': 'NEVADA',
    'PA': 'PENNSYLVANIA',
    'TN': 'TENNESSEE'
}

state_names.values
# for key in list(state_city_dict.keys()):
#     if key in state_names.keys():
#         state_city_dict[state_names[key]] = state_city_dict.values()


state_city_dict = dict((state_names[key],value) for (key,value) in state_city_dict.items())

In [56]:
for key in state_city_dict:
    state_city_dict[key] = list(set(state_city_dict[key]))

In [57]:
state_city_dict

{'ALBERTA': ['ST ALBERT',
  'WEST EDMONTON',
  'ST. ALBERT',
  'DOWNTOWN',
  'SHERWOOD PARK',
  'BEAUMONT',
  'SOUTHEAST EDMONTON',
  'EDMONTON',
  'ENOCH',
  'EAST EDMONTON',
  'EAUX CLAIRES',
  'SHERWOOD',
  'NW EDMONTON',
  'SAINT ALBERT'],
 'ARIZONA': ['TUCSON',
  'SAHUARITA',
  'GREEN VALLEY',
  'DREXEL HEIGHTS',
  'CATALINA',
  'MOUNT LEMMON',
  'CASAS ADOBES',
  'VALENCIA WEST',
  'ORO VALLEY',
  'PICTURE ROCKS',
  'TUSCON',
  'MARANA',
  'CORONA DE TUCSON',
  'SOUTH TUCSON',
  'VAIL'],
 'CALIFORNIA': ['SANTA BARBARA',
  'CARPINTERIA',
  'TRUCKEE',
  'MONTECITO',
  'SUMMERLAND',
  'ISLA VISTA',
  'GOLETA',
  'SANTA  BARBARA'],
 'DELAWARE': ['MONTCHANIN',
  'GREENVILLE',
  'MARSHALLTON',
  'TALLEYVILLE',
  'WILMINGTON MANOR',
  'ELSMERE',
  'WILMINGTON ',
  'STANTON',
  'HOCKESSIN',
  'CHRISTIANA',
  'ASTON',
  'TROLLEY SQUARE',
  'GLEN MILLS',
  'NEW CASTLE',
  'CLAYMONT',
  'PIKE CREEK',
  'NEWARK',
  'WILMINGTON',
  'NEWPORT'],
 'FLORIDA': ['SAN ANTONIO',
  'CARROLLWOOD',
  'S

In [58]:
import json

with open('state_city_dict.json', 'w') as json_file:
    json.dump(state_city_dict, json_file, indent=4)